In [0]:
%run ../Utilities/configuration

In [0]:
%run ../Utilities/common_functions

In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:
import json 
context = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson())
logger = f_get_configured_logger(context)

In [0]:
TABLE_NAME = "transaction"

In [0]:
df_staging = DeltaTable.forName(spark, f"{CATALOG}.{BRONZE_SCHEMA}.{TABLE_NAME}")

In [0]:
try:
    df_select = (df_staging.toDF()
        .select(
            F.col("id").alias("transaction_id"), 
            F.col("customer_id").alias("customer_id"), 
            F.col("date"),
            F.col("market_execution_json"),
            F.col("market_execution_json.source_amount").alias("source_amount"),
            F.col("market_execution_json.destination_amount").alias("destination_amount"),
            F.col("network"),
            F.col("txn_date"),
            F.col("load_timestamp"),
        )
    )
except Exception as e:
    logger.error(e)
    f_push_alert_message(str(e), context)
    raise(e)

In [0]:
try:
    df = (df_select
        .withColumn("transaction_time", F.from_unixtime(F.col("date")).cast("timestamp"))
        .withColumn("execution_plan", F.when(F.col("market_execution_json").getItem("pool").isNotNull(), "pool").otherwise("order_book"))
        .withColumn("pool_id", F.expr("transform(market_execution_json.pool.split_swaps, swap -> swap.swap_route[0].pool_id)"))
        .withColumn("order_book_id", F.expr("transform(market_execution_json.hybrid.SplitSwapResults, swap -> swap.order_book_filled_order_ids[0])"))
        .withColumn("protocol_fee_token",
            F.when(
                F.col("market_execution_json.pool").isNotNull(),
                F.col("market_execution_json.pool.protocol_fees")[0]["token"]
            ).otherwise(F.col("market_execution_json.protocol_fees")[0]["token"])
        )
        .withColumn("protocol_fee_amount",
            F.when(
                F.col("market_execution_json.pool").isNotNull(),
                F.col("market_execution_json.pool.protocol_fees")[0]["amount"]
            ).otherwise(F.col("market_execution_json.protocol_fees")[0]["amount"])
        )
        .withColumn("swap_count",
            F.when(
                F.col("market_execution_json.hybrid").isNotNull(),
                F.size(F.col("market_execution_json.hybrid.SplitSwapResults"))
            ).otherwise(F.size(F.col("market_execution_json.pool.split_swaps")))
        )
    )
except Exception as e:
    logger.error(e)
    f_push_alert_message(str(e), context)
    raise(e)

In [0]:
df = df.select("transaction_id", 
               "customer_id", 
               "transaction_time", 
               "execution_plan", 
               "pool_id", 
               "order_book_id",
               "protocol_fee_token",
               "protocol_fee_amount",
               "swap_count",
               "network",
               "txn_date",
               "load_timestamp"
               )


In [0]:
df.cache()

DataFrame[transaction_id: bigint, customer_id: bigint, transaction_time: timestamp, execution_plan: string, pool_id: array<bigint>, order_book_id: array<bigint>, protocol_fee_token: string, protocol_fee_amount: bigint, swap_count: int, network: string, txn_date: string, load_timestamp: timestamp]

In [0]:
df.take(1)

[Row(transaction_id=1111, customer_id=11101, transaction_time=datetime.datetime(2023, 10, 28, 1, 10, 11), execution_plan='order_book', pool_id=None, order_book_id=[23578333, 23578333, 23578333, 23578333, 23578333, 23578333, 23578333, 23578333, 23578333, 23578333], protocol_fee_token='sol', protocol_fee_amount=7583670275000000, swap_count=10, network='network_1', txn_date='2023-10-28', load_timestamp=datetime.datetime(2024, 11, 6, 5, 51, 23, 24000))]

In [0]:
# Detect merge condition
merge_condition = f_get_primary_key(df)
merge_condition

' tgt.transaction_id=src.transaction_id AND tgt.network=src.network'

In [0]:
# Incremental load
try:
    f_merge_delta_data(df, CATALOG, SILVER_SCHEMA, TABLE_NAME, merge_condition, 'txn_date')
except Exception as e:
    logger.error(e)
    f_push_alert_message(str(e), context)
    raise(e)

In [0]:
df.unpersist()

DataFrame[transaction_id: bigint, customer_id: bigint, transaction_time: timestamp, execution_plan: string, pool_id: array<bigint>, order_book_id: array<bigint>, protocol_fee_token: string, protocol_fee_amount: bigint, swap_count: int, network: string, txn_date: string, load_timestamp: timestamp]

In [0]:
dbutils.notebook.exit("Success")